In [1]:
"""
Inception模块（Going Deeper with Convolutions）

Inception模块的设计理念：
在同一层级上使用不同尺寸的卷积核并行处理，然后将结果拼接，
使网络能够同时捕获不同尺度的特征。

核心优势：
1. 多尺度特征提取：1x1、3x3、5x5卷积核同时工作
2. 降维减少计算量：1x1卷积降低通道数
3. 增加网络宽度和深度：在相同计算预算下提升性能
4. 自动学习最优特征组合：网络决定每个尺度的权重

本示例实现Inception v1（GoogLeNet）中的标准模块
"""

from keras import layers
from keras.models import Model
from keras.layers import Input
import numpy as np

# 定义输入特征图
input_tensor = Input(shape=(35, 35, 256))
x = input_tensor

# 分支A：1x1卷积，捕获点特征
branch_a = layers.Conv2D(64, 1, activation='relu', padding='same')(x)

# 分支B：1x1卷积降维 + 3x3卷积
# 先降维再卷积，减少计算量
branch_b = layers.Conv2D(96, 1, activation='relu', padding='same')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', padding='same')(branch_b)

# 分支C：1x1卷积降维 + 5x5卷积
# 捕获更大感受野的特征
branch_c = layers.Conv2D(16, 1, activation='relu', padding='same')(x)
branch_c = layers.Conv2D(32, 5, activation='relu', padding='same')(branch_c)

# 分支D：3x3最大池化 + 1x1卷积
# 保留显著特征的同时降维
branch_d = layers.MaxPooling2D(3, strides=1, padding='same')(x)
branch_d = layers.Conv2D(32, 1, activation='relu', padding='same')(branch_d)

# 沿通道维度拼接所有分支：64 + 128 + 32 + 32 = 256个通道
output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)

# 构建模型
model = Model(inputs=input_tensor, outputs=output)
model.summary()

# 验证可运行性
test_input = np.random.randn(1, 35, 35, 256).astype('float32')
output_result = model.predict(test_input, verbose=0)
print(f"\n输入shape: {test_input.shape}")
print(f"输出shape: {output_result.shape}")
print(f"通道数: {256} -> {output_result.shape[-1]}")
print(f"空间尺寸保持不变: {35}x{35}")
print("Inception模块验证成功")

2025-12-12 22:20:53.756294: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-12 22:20:53.762341: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-12 22:20:53.769951: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-12 22:20:53.772197: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-12 22:20:53.777944: I tensorflow/core/platform/cpu_feature_guar

I0000 00:00:1765549254.960132   34185 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1765549254.982101   34185 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1765549254.983124   34185 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1765549254.985507   34185 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 35, 35,    │          0 │ -                 │
│ (InputLayer)        │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 35, 35,    │     24,672 │ input_layer[0][0] │
│                     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 35, 35,    │      4,112 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 35, 35,    │          0 │ input_layer[0][0] │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 35, 35,    │     16,448 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 35, 35,    │    110,720 │ conv2d_1[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 35, 35,    │     12,832 │ conv2d_3[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 35, 35,    │      8,224 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 35, 35,    │          0 │ conv2d[0][0],     │
│ (Concatenate)       │ 256)              │            │ conv2d_2[0][0],   │
│                     │                   │            │ conv2d_4[0][0],   │
│                     │                   │            │ conv2d_5[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 177,008 (691.44 KB)

 Trainable params: 177,008 (691.44 KB)

 Non-trainable params: 0 (0.00 B)

I0000 00:00:1765549255.572678   34337 service.cc:146] XLA service 0x7f050c006280 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1765549255.572700   34337 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4080 Laptop GPU, Compute Capability 8.9
2025-12-12 22:20:55.578960: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-12-12 22:20:55.594518: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 91301



输入shape: (1, 35, 35, 256)
输出shape: (1, 35, 35, 256)
通道数: 256 -> 256
空间尺寸保持不变: 35x35
Inception模块验证成功


I0000 00:00:1765549255.962886   34337 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
